# import modules

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
# from sklearn.pipeline import Pipeline
import datetime
import time

# Data set 불러오기

In [2]:
# 여러분의 폴더가 저장되어있는 위치 경로를 써 주세요
# 그리고 데이터파일(주최측에서 제공한) 의 이름을 Train set 으로 변셩해 주세요
# 한인욱 directory = 'C:/Users/goran/Desktop/Git_Hub/Project/20S_Bigcon'
# 김정현 directory = 'C:/Users/Coby/Desktop/20S_Bigcon/20S_Bigcon'
# 신예진 directory = 'C:/Users/Rstudio/Desktop/Bigcon2020/20S_Bigcon'
# 이규민 directory = 'C:/Statistics/20S_Bigcon'
# 최정욱 directory = 'C:/Users/lg/Desktop/Bigcon2020'
directory = 'C:/Users/goran/Desktop/Git_Hub/Project/20S_Bigcon'

In [3]:
# 그러면 이렇게 그 폴더 안의 Train set.xlsx 을 잘 읽어옵니다! 
X_df=pd.read_excel(directory+'/Train set.xlsx',header = 1) 

In [4]:
# 데이터 reset 하고싶으면 위의 셀 말고 이거 돌려가면서 reset 하세요! 그게 제일 빨라요
X = X_df.copy()

In [5]:
X.head() 

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0
1,2019-01-01 06:00:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0
3,2019-01-01 06:20:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0


# 결측치 처리 및 변수늘리기

In [6]:
X.isnull().sum()

방송일시         0
노출(분)    16784
마더코드         0
상품코드         0
상품명          0
상품군          0
판매단가         0
취급액        937
dtype: int64

## 노출(분) 결측치 처리

노출분은 같은 시간에 같이 방송되어서 NaN 으로 뜨는것이다. <br>
아마 같은 시간에 방송되어진 값을 평균해서 채우면 될듯?

In [7]:
X.loc[:,'평균방송분'] = X.loc[:,'노출(분)']

In [8]:
# 0 노출시간이 0 인 데이터가 없다.
# 즉 Na 처리할떄 0으로 대채한 후에 평균떄려도 아무 상관 없다는 이야기!
(X.loc[:,'노출(분)']==0).sum()

0

In [9]:
# 노출(분) 을 제일 먼저 시작한 시간의 값으로 만드는 FUNCTION 정의
# 아직 맞는지는 잘 몰라서 안맞는 경우 수정해주세요.
def NA_minute(df=X):
    # 곁측치를 0으로 대체
    df['노출(분)'] = df['노출(분)'].fillna(0)
    
    count_0 = [] 
    count_not0 = [] # 0이 아닌 수의 index
    for idx in df.index :
        if df.loc[idx,'노출(분)'] == 0 :
            count_0.append(idx)
        else :
            count_not0.append(idx)
    count_not0.append(len(df)) # 맨 마지막 열에 0 인 경우를 대비하여

    for idx,x in enumerate(count_not0):
        if idx != len(count_not0)-1: 
            if (count_not0[idx+1] - count_not0[idx] - 1) != 0:
                start = count_not0[idx] 
                end = count_not0[idx+1]
                num = end - start 
                df.loc[start:(end-1),'노출(분)'] = df.loc[start,'노출(분)'] # / num 을 붙여주면 평균으로 만들게 됩니다.
    return df

In [10]:
X = NA_minute(X) ; X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균방송분
0,2019-01-01 06:00:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,20.000000
1,2019-01-01 06:00:00,20.000000,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,NaN
2,2019-01-01 06:20:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,20.000000
3,2019-01-01 06:20:00,20.000000,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,NaN
4,2019-01-01 06:40:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,20.000000
...,...,...,...,...,...,...,...,...,...
38304,2020-01-01 00:20:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,20.000000
38305,2020-01-01 00:40:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,20.000000
38306,2020-01-01 01:00:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,20.000000
38307,2020-01-01 01:20:00,20.000000,100490,201478,더케이 예다함 상조서비스(티포트),무형,0,NaN,20.000000


## 얼마나 같은 방송일시에 겹쳐있는지 변수늘리기

In [11]:
X.loc[:,'겹치는수'] = 1

In [12]:
for idx in range(1,len(X))  :
    if ~(X.loc[idx-1,'마더코드'] == X.loc[idx,'마더코드']): # 앞과 마더코드가 다를때!
        continue # 아무 관련 없으므로 다음 for 문으로 넘어가도록 합시다
    elif (X.loc[idx-1,'마더코드'] == X.loc[idx,'마더코드']) : # 앞 구간과 같아버리면 같은게 지속되고있는거겟죠!
        if X.loc[idx-1,'방송일시'] == X.loc[idx,'방송일시']: # 만약 전의 방송일시와 지금이 일치하면 그냥 전 index 의 겹치는 수를 씁니다. 
            X.loc[idx,'겹치는수'] = X.loc[idx-1,'겹치는수']
        elif ~(X.loc[idx-1,'방송일시'] == X.loc[idx,'방송일시']): # 만약 전의 방송일시와 지금이 다르다면 , 마더코드는 같은데 방송일시가 다르단거겟죠? index 를 증가시킵니다.
            X.loc[idx,'겹치는수'] = X.loc[idx-1,'겹치는수']+1

In [13]:
X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균방송분,겹치는수
0,2019-01-01 06:00:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,20.000000,1
1,2019-01-01 06:00:00,20.000000,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,NaN,1
2,2019-01-01 06:20:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,20.000000,2
3,2019-01-01 06:20:00,20.000000,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,NaN,2
4,2019-01-01 06:40:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,20.000000,3
...,...,...,...,...,...,...,...,...,...,...
38304,2020-01-01 00:20:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,20.000000,1
38305,2020-01-01 00:40:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,20.000000,2
38306,2020-01-01 01:00:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,20.000000,3
38307,2020-01-01 01:20:00,20.000000,100490,201478,더케이 예다함 상조서비스(티포트),무형,0,NaN,20.000000,1


## 평균 방송분 변수늘리기

In [14]:
def MEAN_minute(df=X):
    # 곁측치를 0으로 대체
    df['평균방송분'] = df['평균방송분'].fillna(0)
    
    count_0 = [] 
    count_not0 = [] # 0이 아닌 수의 index
    for idx in df.index :
        if df.loc[idx,'평균방송분'] == 0 :
            count_0.append(idx)
        else :
            count_not0.append(idx)
    count_not0.append(len(df)) # 맨 마지막 열에 0 인 경우를 대비하여

    for idx,x in enumerate(count_not0):
        if idx != len(count_not0)-1: 
            if (count_not0[idx+1] - count_not0[idx] - 1) != 0:
                start = count_not0[idx] 
                end = count_not0[idx+1]
                num = end - start 
                df.loc[start:(end-1),'평균방송분'] = df.loc[start,'평균방송분'] / num
    return df

In [15]:
X = MEAN_minute(X) ; X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균방송분,겹치는수
0,2019-01-01 06:00:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,10.000000,1
1,2019-01-01 06:00:00,20.000000,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,10.000000,1
2,2019-01-01 06:20:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,10.000000,2
3,2019-01-01 06:20:00,20.000000,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,10.000000,2
4,2019-01-01 06:40:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,10.000000,3
...,...,...,...,...,...,...,...,...,...,...
38304,2020-01-01 00:20:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,20.000000,1
38305,2020-01-01 00:40:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,20.000000,2
38306,2020-01-01 01:00:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,20.000000,3
38307,2020-01-01 01:20:00,20.000000,100490,201478,더케이 예다함 상조서비스(티포트),무형,0,NaN,20.000000,1


## 상품군 결측치 처리

 판매단가가 0 인 상품은 예측에서 제외

In [16]:
def NA_goods(df=X):
    df=df.loc[df['판매단가'] != 0,:]
    df = df.reset_index(drop=True)
    return df

In [17]:
X = NA_goods(X) ; X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균방송분,겹치는수
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,10.0,1
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,10.0,1
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,10.0,2
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,10.0,2
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,10.0,3
...,...,...,...,...,...,...,...,...,...,...
37367,2019-12-31 23:40:00,20.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,5.0,2
37368,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,5.0,3
37369,2020-01-01 00:00:00,20.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,5.0,3
37370,2020-01-01 00:00:00,20.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,5.0,3


##  토요일 곁측치 처리

매주 토요일 18:00~18:20은 정보방송시간으로 추정 제외(당사사정에 따라 ±20 편성 변경 있음)

In [18]:
def NA_Saturday(df = X):
    t = ['월','화','수','목','금','토','일']
    r = datetime.datetime.today()
    def printDayOfTheWeek(year, month, day):
        dayOfTheWeek = ['월','화','수','목','금','토','일']
        return dayOfTheWeek[datetime.datetime(year,month,day).weekday()]
    day_list = []
    for date in X.iloc[:,0]:
        date = str(date)
        year = int(date[:4])
        month = int(date[5:7])
        day = int(date[8:10])
        first = printDayOfTheWeek(year,month,day)
        second = date[11:16]
        day_list.append(first+second)
    X["요일/시간"] = day_list
    for i in range(len(df)):
        time = df.loc[i,'요일/시간'] # 여기도 요일/시간 인 loc 로 바꾸었습니다!
        if (time[0:3]=="토18" and int(time[4:])<20):
            df = df.drop(i)
    return df

In [19]:
NA_Saturday(X)

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균방송분,겹치는수,요일/시간
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,10.0,1,화06:00
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,10.0,1,화06:00
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,10.0,2,화06:20
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,10.0,2,화06:20
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,10.0,3,화06:40
...,...,...,...,...,...,...,...,...,...,...,...
37367,2019-12-31 23:40:00,20.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,5.0,2,화23:40
37368,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,5.0,3,수00:00
37369,2020-01-01 00:00:00,20.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,5.0,3,수00:00
37370,2020-01-01 00:00:00,20.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,5.0,3,수00:00


In [20]:
#check 하는 용도라 뺴놨습니다!
for i in range(len(NA_Saturday(X))):
    time = X.loc[i,'요일/시간'] # 여기 loc 로 바꾸었습니다. 왜냐하면 앞에서 평균방송분 열을 추가해야되서 열이 하나 늘어났어요
    if time[0:3]=="토18":
        print(time)

토18:20
토18:40
토18:20
토18:40
토18:20
토18:40
토18:20
토18:40
토18:20
토18:40
토18:20
토18:40
토18:20
토18:40
토18:20
토18:40
토18:20
토18:40
토18:20
토18:40
토18:20
토18:30
토18:40
토18:50
토18:20
토18:50
토18:20
토18:30
토18:40
토18:50
토18:20
토18:40
토18:20
토18:40
토18:20
토18:40
토18:20
토18:40
토18:20
토18:20
토18:20
토18:20
토18:20
토18:20
토18:40
토18:40
토18:40
토18:40
토18:40
토18:40


In [21]:
X = NA_Saturday(X) ; X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균방송분,겹치는수,요일/시간
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,10.0,1,화06:00
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,10.0,1,화06:00
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,10.0,2,화06:20
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,10.0,2,화06:20
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,10.0,3,화06:40
...,...,...,...,...,...,...,...,...,...,...,...
37367,2019-12-31 23:40:00,20.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,5.0,2,화23:40
37368,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,5.0,3,수00:00
37369,2020-01-01 00:00:00,20.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,5.0,3,수00:00
37370,2020-01-01 00:00:00,20.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,5.0,3,수00:00


힘들게 코드를 짰으나 토요일 18:00~18:20에 해당하는 방송은 없었다.............................

## 취급액 50000-> 0 처리

In [22]:
(X.loc[:,'취급액'] <= 50000).sum()

1993

In [23]:
def NA_50000(df=X):
    df.loc[df.loc[:,'취급액'] <= 50000 ,'취급액'] = 0
    return df

In [24]:
X = NA_50000(X) ; X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균방송분,겹치는수,요일/시간
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,10.0,1,화06:00
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,10.0,1,화06:00
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,10.0,2,화06:20
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,10.0,2,화06:20
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,10.0,3,화06:40
...,...,...,...,...,...,...,...,...,...,...,...
37367,2019-12-31 23:40:00,20.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,5.0,2,화23:40
37368,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,5.0,3,수00:00
37369,2020-01-01 00:00:00,20.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,5.0,3,수00:00
37370,2020-01-01 00:00:00,20.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,5.0,3,수00:00


# 변수 늘리기

## 시간 데이터 늘리기

In [25]:
# 시각은 우선 시,분을 합쳐서 하나의 값으로 만들었습니다.
def time_feature(df = X):
    df['방송일시']=df['방송일시'].astype('str')
    df['날짜']=df['방송일시'].str.split(' ').str[0]
    df['시간']=df['방송일시'].str.split(' ').str[1]
    df['년']=df['날짜'].str.split('-').str[0]
    df['월']=df['날짜'].str.split('-').str[1]
    df['일']=df['날짜'].str.split('-').str[2]
    df['시']=df['시간'].str.split(':').str[0]
    df['분']=df['시간'].str.split(':').str[1]
    df['초']=df['시간'].str.split(':').str[2]
    df['시각'] = df['시'].astype('float32') + df['분'].astype('float32')*(1/60)
    df['시분'] = df['시'] + ':' + df['분']
    df.drop(['시','분','초','시간'],axis=1,inplace = True)
    return(df)

In [26]:
X = time_feature(X) ; X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균방송분,겹치는수,요일/시간,날짜,년,월,일,시각,시분
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,10.0,1,화06:00,2019-01-01,2019,01,01,6.000000,06:00
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,10.0,1,화06:00,2019-01-01,2019,01,01,6.000000,06:00
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,10.0,2,화06:20,2019-01-01,2019,01,01,6.333333,06:20
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,10.0,2,화06:20,2019-01-01,2019,01,01,6.333333,06:20
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,10.0,3,화06:40,2019-01-01,2019,01,01,6.666667,06:40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37367,2019-12-31 23:40:00,20.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,5.0,2,화23:40,2019-12-31,2019,12,31,23.666666,23:40
37368,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,5.0,3,수00:00,2020-01-01,2020,01,01,0.000000,00:00
37369,2020-01-01 00:00:00,20.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,5.0,3,수00:00,2020-01-01,2020,01,01,0.000000,00:00
37370,2020-01-01 00:00:00,20.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,5.0,3,수00:00,2020-01-01,2020,01,01,0.000000,00:00


## 식사 시간과 관련된 변수

식사시간의 경우 유의미한 데이터가 있을 수 있다. 그 시간대를 잡아서 변수를 늘리면 효과가 있을거같다. <br>
하지만 아침 식사시간의 경우 거르는 인구가 33%(통계청) 이나 되는 이유로, 오히려 혼란을 가중시킬 수 있기떄문에 그냥 점심과 저녁만 고려하자

In [27]:
def eat_feature(df = X):
    df['점심'] = 0
    df.loc[ (12<=df['시각']) & (df['시각'] <= 13.5),'점심']  = 1
    df['저녁'] = 0
    df.loc[ (18.5<=df['시각'])&(df['시각']<=20.0) , '저녁'] = 1
    return(df)

In [28]:
X = eat_feature(X) ; X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균방송분,겹치는수,요일/시간,날짜,년,월,일,시각,시분,점심,저녁
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,10.0,1,화06:00,2019-01-01,2019,01,01,6.000000,06:00,0,0
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,10.0,1,화06:00,2019-01-01,2019,01,01,6.000000,06:00,0,0
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,10.0,2,화06:20,2019-01-01,2019,01,01,6.333333,06:20,0,0
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,10.0,2,화06:20,2019-01-01,2019,01,01,6.333333,06:20,0,0
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,10.0,3,화06:40,2019-01-01,2019,01,01,6.666667,06:40,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37367,2019-12-31 23:40:00,20.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,5.0,2,화23:40,2019-12-31,2019,12,31,23.666666,23:40,0,0
37368,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,5.0,3,수00:00,2020-01-01,2020,01,01,0.000000,00:00,0,0
37369,2020-01-01 00:00:00,20.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,5.0,3,수00:00,2020-01-01,2020,01,01,0.000000,00:00,0,0
37370,2020-01-01 00:00:00,20.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,5.0,3,수00:00,2020-01-01,2020,01,01,0.000000,00:00,0,0


In [29]:
pd.Series(X['점심']).value_counts() # 좀 sparse 하긴 한데 의미는 있을거라고 믿는다...

0    34415
1     2957
Name: 점심, dtype: int64

## 강수량 변수

날씨 데이터의 경우 기상청의 공공데이터를 사용했다.
일별 기온(평균기온, 최고기온, 최저기온), 일별 강수량, 월별 기상정보(폭염일수, 열대야일수)를 고려했다.

In [30]:
rain_df=pd.read_csv(directory+'/19rain.csv',header = 7, encoding='CP949')
rain = rain_df.copy()

In [31]:
rain.head()

,날짜,지점,강수량(mm)
0,2019-01-01,전국,0.1
1,2019-01-02,전국,0.0
2,2019-01-03,전국,0.0
3,2019-01-04,전국,0.0
4,2019-01-05,전국,0.0


In [32]:
def rain_feature(df = X):
       
    df = pd.merge(df, rain, on="날짜", how= "inner")
    df.drop(['지점'],axis=1,inplace = True)
    return(df)

X = rain_feature(X) ; X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균방송분,겹치는수,요일/시간,날짜,년,월,일,시각,시분,점심,저녁,강수량(mm)
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,10.0,1,화06:00,2019-01-01,2019,01,01,6.000000,06:00,0,0,0.1
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,10.0,1,화06:00,2019-01-01,2019,01,01,6.000000,06:00,0,0,0.1
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,10.0,2,화06:20,2019-01-01,2019,01,01,6.333333,06:20,0,0,0.1
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,10.0,2,화06:20,2019-01-01,2019,01,01,6.333333,06:20,0,0,0.1
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,10.0,3,화06:40,2019-01-01,2019,01,01,6.666667,06:40,0,0,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37367,2019-12-31 23:40:00,20.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,5.0,2,화23:40,2019-12-31,2019,12,31,23.666666,23:40,0,0,0.1
37368,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,5.0,3,수00:00,2020-01-01,2020,01,01,0.000000,00:00,0,0,0.0
37369,2020-01-01 00:00:00,20.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,5.0,3,수00:00,2020-01-01,2020,01,01,0.000000,00:00,0,0,0.0
37370,2020-01-01 00:00:00,20.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,5.0,3,수00:00,2020-01-01,2020,01,01,0.000000,00:00,0,0,0.0


## 온도 변수

In [33]:
temp_df=pd.read_csv(directory+'/19temp.csv',header = 7, encoding='CP949')
temp = temp_df.copy()

In [34]:
temp.head()

,날짜,지점,평균기온(℃),최저기온(℃),최고기온(℃)
0,2019-01-01,전국,-2.1,-5.8,2.1
1,2019-01-02,전국,-2.5,-7.0,3.2
2,2019-01-03,전국,-2.1,-7.7,5.0
3,2019-01-04,전국,-0.7,-7.3,5.3
4,2019-01-05,전국,0.2,-4.6,5.3


In [35]:
def temp_feature(df = X):
    df = pd.merge(df, temp, on="날짜", how= "inner")
    df.drop(['지점'],axis=1,inplace = True)
    return(df)
X = temp_feature(X) ; X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균방송분,겹치는수,...,월,일,시각,시분,점심,저녁,강수량(mm),평균기온(℃),최저기온(℃),최고기온(℃)
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,10.0,1,...,01,01,6.000000,06:00,0,0,0.1,-2.1,-5.8,2.1
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,10.0,1,...,01,01,6.000000,06:00,0,0,0.1,-2.1,-5.8,2.1
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,10.0,2,...,01,01,6.333333,06:20,0,0,0.1,-2.1,-5.8,2.1
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,10.0,2,...,01,01,6.333333,06:20,0,0,0.1,-2.1,-5.8,2.1
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,10.0,3,...,01,01,6.666667,06:40,0,0,0.1,-2.1,-5.8,2.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37367,2019-12-31 23:40:00,20.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,5.0,2,...,12,31,23.666666,23:40,0,0,0.1,-3.8,-6.7,0.4
37368,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,5.0,3,...,01,01,0.000000,00:00,0,0,0.0,4.1,-6.5,4.1
37369,2020-01-01 00:00:00,20.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,5.0,3,...,01,01,0.000000,00:00,0,0,0.0,4.1,-6.5,4.1
37370,2020-01-01 00:00:00,20.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,5.0,3,...,01,01,0.000000,00:00,0,0,0.0,4.1,-6.5,4.1


## 연예인 변수 

In [36]:
def celebrity(df=X):
    df['연예인']=0
    list_celebrity=["전지현", "팽현숙", "전철우", "강레오", "김선영", "김정배",
                    "김규흔", "이봉원", "오세득", "유귀열", "이경제", "이만기",
                    "이보은", "임성근", "전철우", "최인선", "김병만", "김병지", 
                    "이동수", "서장훈", "송도순", "효재", "천수봉", 
                    "김정문", "숀리", "이정섭", "황성주"]
    for i in list_celebrity:
        df.loc[df.상품명.str.contains(i),'연예인']=1
    return df

In [37]:
X=celebrity(X); X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균방송분,겹치는수,...,일,시각,시분,점심,저녁,강수량(mm),평균기온(℃),최저기온(℃),최고기온(℃),연예인
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,10.0,1,...,01,6.000000,06:00,0,0,0.1,-2.1,-5.8,2.1,0
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,10.0,1,...,01,6.000000,06:00,0,0,0.1,-2.1,-5.8,2.1,0
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,10.0,2,...,01,6.333333,06:20,0,0,0.1,-2.1,-5.8,2.1,0
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,10.0,2,...,01,6.333333,06:20,0,0,0.1,-2.1,-5.8,2.1,0
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,10.0,3,...,01,6.666667,06:40,0,0,0.1,-2.1,-5.8,2.1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37367,2019-12-31 23:40:00,20.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,5.0,2,...,31,23.666666,23:40,0,0,0.1,-3.8,-6.7,0.4,0
37368,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,5.0,3,...,01,0.000000,00:00,0,0,0.0,4.1,-6.5,4.1,0
37369,2020-01-01 00:00:00,20.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,5.0,3,...,01,0.000000,00:00,0,0,0.0,4.1,-6.5,4.1,0
37370,2020-01-01 00:00:00,20.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,5.0,3,...,01,0.000000,00:00,0,0,0.0,4.1,-6.5,4.1,0


In [38]:
print(X.sum().연예인) #연예인 이름을 사용한 총 데이터 수

924


## 월급날

In [39]:
#https://news.sbs.co.kr/news/endPage.do?news_id=N1005505452
#https://m.blog.naver.com/youngkbblog/221714986107

In [40]:
def income(df = X):
    df['월급날'] = 0
    df.loc[ ((10<=df['일'].astype('float32'))&(df['일'].astype('float32')<=13)) | ((20<=df['일'].astype('float32'))&(df['일'].astype('float32')<= 23)) | ((25<=df['일'].astype('float32')) & (df['일'].astype('float32')<= 28)),'월급날']  = 1
    return(df)

In [41]:
X=income(X);X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균방송분,겹치는수,...,시각,시분,점심,저녁,강수량(mm),평균기온(℃),최저기온(℃),최고기온(℃),연예인,월급날
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,10.0,1,...,6.000000,06:00,0,0,0.1,-2.1,-5.8,2.1,0,0
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,10.0,1,...,6.000000,06:00,0,0,0.1,-2.1,-5.8,2.1,0,0
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,10.0,2,...,6.333333,06:20,0,0,0.1,-2.1,-5.8,2.1,0,0
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,10.0,2,...,6.333333,06:20,0,0,0.1,-2.1,-5.8,2.1,0,0
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,10.0,3,...,6.666667,06:40,0,0,0.1,-2.1,-5.8,2.1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37367,2019-12-31 23:40:00,20.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,5.0,2,...,23.666666,23:40,0,0,0.1,-3.8,-6.7,0.4,0,0
37368,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,5.0,3,...,0.000000,00:00,0,0,0.0,4.1,-6.5,4.1,0,0
37369,2020-01-01 00:00:00,20.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,5.0,3,...,0.000000,00:00,0,0,0.0,4.1,-6.5,4.1,0,0
37370,2020-01-01 00:00:00,20.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,5.0,3,...,0.000000,00:00,0,0,0.0,4.1,-6.5,4.1,0,0


In [42]:
X.loc[X["일"]=="10", ["월급날"]] #잘 들어갔는지 확인
print(X.sum().월급날) #총 월급날

14313


## 금,토 및 공휴일

In [43]:
#https://0muwon.com/entry/2019%EB%85%84-%ED%9C%B4%EC%9D%BC-%EA%B3%B5%ED%9C%B4%EC%9D%BC-%EB%8C%80%EC%B2%B4%ED%9C%B4%EC%9D%BC-%ED%99%A9%EA%B8%88%EC%97%B0%ED%9C%B4-%EC%9B%94%EB%B3%84-%EC%99%84%EB%B2%BD%EC%A0%95%EB%A6%AC

In [44]:
def holiday(df = X):
    df["휴일"] = 0
    holiday_list=[[1,4,5,6,11,12,13,18,19,20,25,26,27],
                  [1,2,3,4,5,6,8,9,10,15,16,17,22,23,24],
                  [1,2,3,8,9,10,15,16,17,22,23,24,29,30,31],
                  [5,6,7,12,13,14,19,20,21,26,27,28],
                  [3,4,5,6,10,11,12,17,18,19,24,25,26,31],
                  [1,2,6,7,8,9,14,15,16,21,22,23,28,29,30],
                  [5,6,7,12,13,14,19,20,21,26,27,28],
                  [2,3,4,9,10,11,15,16,17,18,23,24,25,30,31],
                  [1,6,7,8,12,13,14,15,20,21,22,27,28,29],
                  [3,4,5,6,9,11,12,13,18,19,20,25,26,27],
                  [1,2,3,8,9,10,15,16,17,22,23,24,29,30],
                  [1,6,7,8,13,14,15,20,21,22,25,27,28,29]]
    for i, holiday in enumerate(holiday_list):
        for j in holiday:
            df.loc[ (df["월"].astype("float32")==i+1) & (df["일"].astype("float32")==j),'휴일'] = 1
    return(df)

In [45]:
X=holiday(X);X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균방송분,겹치는수,...,시분,점심,저녁,강수량(mm),평균기온(℃),최저기온(℃),최고기온(℃),연예인,월급날,휴일
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,10.0,1,...,06:00,0,0,0.1,-2.1,-5.8,2.1,0,0,1
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,10.0,1,...,06:00,0,0,0.1,-2.1,-5.8,2.1,0,0,1
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,10.0,2,...,06:20,0,0,0.1,-2.1,-5.8,2.1,0,0,1
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,10.0,2,...,06:20,0,0,0.1,-2.1,-5.8,2.1,0,0,1
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,10.0,3,...,06:40,0,0,0.1,-2.1,-5.8,2.1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37367,2019-12-31 23:40:00,20.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,5.0,2,...,23:40,0,0,0.1,-3.8,-6.7,0.4,0,0,0
37368,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,5.0,3,...,00:00,0,0,0.0,4.1,-6.5,4.1,0,0,1
37369,2020-01-01 00:00:00,20.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,5.0,3,...,00:00,0,0,0.0,4.1,-6.5,4.1,0,0,1
37370,2020-01-01 00:00:00,20.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,5.0,3,...,00:00,0,0,0.0,4.1,-6.5,4.1,0,0,1


In [46]:
X.loc[(X["월"].astype("float32")==3) & (X["일"].astype("float32")==1),["휴일"]] #잘 들어갔는지 확인
print(X.sum().휴일) #총 휴일

17161


## 상품군 power

In [47]:
def goods_power(df = X):
    df["상품군power"] = 0
    for name in ('가구','가전','건강기능','농수축','생활용품','속옷','의류','이미용','잡화','주방','침구'):
        revenue = 0
        runtime = 0
        for i in range(len(df)):
            if df.loc[i,'상품군'] == name:
                revenue += df.loc[i,'취급액']/df.loc[i,'판매단가']
                runtime += df.loc[i,'평균방송분']
            else:
                pass
        for i in range(len(df)):
            if df.loc[i,'상품군'] == name:
                df.loc[i,'상품군power'] = round(revenue/runtime,2)
    
    return(df)

In [48]:
X = goods_power(X) ; X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균방송분,겹치는수,...,점심,저녁,강수량(mm),평균기온(℃),최저기온(℃),최고기온(℃),연예인,월급날,휴일,상품군power
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,10.0,1,...,0,0,0.1,-2.1,-5.8,2.1,0,0,1,26.38
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,10.0,1,...,0,0,0.1,-2.1,-5.8,2.1,0,0,1,26.38
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,10.0,2,...,0,0,0.1,-2.1,-5.8,2.1,0,0,1,26.38
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,10.0,2,...,0,0,0.1,-2.1,-5.8,2.1,0,0,1,26.38
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,10.0,3,...,0,0,0.1,-2.1,-5.8,2.1,0,0,1,26.38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37367,2019-12-31 23:40:00,20.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,5.0,2,...,0,0,0.1,-3.8,-6.7,0.4,0,0,0,19.35
37368,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,5.0,3,...,0,0,0.0,4.1,-6.5,4.1,0,0,1,19.35
37369,2020-01-01 00:00:00,20.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,5.0,3,...,0,0,0.0,4.1,-6.5,4.1,0,0,1,19.35
37370,2020-01-01 00:00:00,20.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,5.0,3,...,0,0,0.0,4.1,-6.5,4.1,0,0,1,19.35


## 소비자물가지수

In [49]:
def CPI(df = X):
    df['CPI'] = 0
    day_list = []
    for date in df.loc[:,'방송일시']:
        year = date[2:4]
        month = date[5:7]
        day_list.append(year+month)
    df['연월'] = day_list
    cpi1,cpi2,cpi3,cpi4,cpi5,cpi6,cpi7,cpi8,cpi9,cpi10,cpi11,cpi12,cpi13 = 0.8, 0.5, 0.4, 0.6, 0.7, 0.7, 0.6, 0.0, -0.4, 0.0, 0.2, 0.7, 1.5
    df.loc[df['연월']== '1901','CPI'] = cpi1
    df.loc[df['연월']== '1902','CPI'] = cpi2
    df.loc[df['연월']== '1903','CPI'] = cpi3
    df.loc[df['연월']== '1904','CPI'] = cpi4
    df.loc[df['연월']== '1905','CPI'] = cpi5
    df.loc[df['연월']== '1906','CPI'] = cpi6
    df.loc[df['연월']== '1907','CPI'] = cpi7
    df.loc[df['연월']== '1908','CPI'] = cpi8
    df.loc[df['연월']== '1909','CPI'] = cpi9
    df.loc[df['연월']== '1910','CPI'] = cpi10
    df.loc[df['연월']== '1911','CPI'] = cpi11
    df.loc[df['연월']== '1912','CPI'] = cpi12
    df.loc[df['연월']== '2001','CPI'] = cpi13
    del df['연월']
    
    return(df)

In [50]:
X = CPI(X); X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균방송분,겹치는수,...,저녁,강수량(mm),평균기온(℃),최저기온(℃),최고기온(℃),연예인,월급날,휴일,상품군power,CPI
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,10.0,1,...,0,0.1,-2.1,-5.8,2.1,0,0,1,26.38,0.8
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,10.0,1,...,0,0.1,-2.1,-5.8,2.1,0,0,1,26.38,0.8
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,10.0,2,...,0,0.1,-2.1,-5.8,2.1,0,0,1,26.38,0.8
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,10.0,2,...,0,0.1,-2.1,-5.8,2.1,0,0,1,26.38,0.8
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,10.0,3,...,0,0.1,-2.1,-5.8,2.1,0,0,1,26.38,0.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37367,2019-12-31 23:40:00,20.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,5.0,2,...,0,0.1,-3.8,-6.7,0.4,0,0,0,19.35,0.7
37368,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,5.0,3,...,0,0.0,4.1,-6.5,4.1,0,0,1,19.35,1.5
37369,2020-01-01 00:00:00,20.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,5.0,3,...,0,0.0,4.1,-6.5,4.1,0,0,1,19.35,1.5
37370,2020-01-01 00:00:00,20.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,5.0,3,...,0,0.0,4.1,-6.5,4.1,0,0,1,19.35,1.5


## 소비자 동향지수

In [51]:
def CSI(df = X):
    df['CSI'] = 0
    day_list = []
    for date in df.loc[:,'방송일시']:
        year = date[2:4]
        month = date[5:7]
        day_list.append(year+month)
    df['연월'] = day_list
    csi1,csi2,csi3,csi4,csi5,csi6,csi7,csi8,csi9,csi10,csi11,csi12,csi13 = 97.5, 99.5, 99.8, 101.6, 97.9, 97.5, 95.9, 92.5, 96.9, 98.6, 101.0, 100.5, 104.2
    df.loc[df['연월']== '1901','CSI'] = csi1
    df.loc[df['연월']== '1902','CSI'] = csi2
    df.loc[df['연월']== '1903','CSI'] = csi3
    df.loc[df['연월']== '1904','CSI'] = csi4
    df.loc[df['연월']== '1905','CSI'] = csi5
    df.loc[df['연월']== '1906','CSI'] = csi6
    df.loc[df['연월']== '1907','CSI'] = csi7
    df.loc[df['연월']== '1908','CSI'] = csi8
    df.loc[df['연월']== '1909','CSI'] = csi9
    df.loc[df['연월']== '1910','CSI'] = csi10
    df.loc[df['연월']== '1911','CSI'] = csi11
    df.loc[df['연월']== '1912','CSI'] = csi12
    df.loc[df['연월']== '2001','CSI'] = csi13
    del df['연월']
    
    return(df)

In [52]:
X = CSI(X); X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균방송분,겹치는수,...,강수량(mm),평균기온(℃),최저기온(℃),최고기온(℃),연예인,월급날,휴일,상품군power,CPI,CSI
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,10.0,1,...,0.1,-2.1,-5.8,2.1,0,0,1,26.38,0.8,97.5
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,10.0,1,...,0.1,-2.1,-5.8,2.1,0,0,1,26.38,0.8,97.5
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,10.0,2,...,0.1,-2.1,-5.8,2.1,0,0,1,26.38,0.8,97.5
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,10.0,2,...,0.1,-2.1,-5.8,2.1,0,0,1,26.38,0.8,97.5
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,10.0,3,...,0.1,-2.1,-5.8,2.1,0,0,1,26.38,0.8,97.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37367,2019-12-31 23:40:00,20.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,5.0,2,...,0.1,-3.8,-6.7,0.4,0,0,0,19.35,0.7,100.5
37368,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,5.0,3,...,0.0,4.1,-6.5,4.1,0,0,1,19.35,1.5,104.2
37369,2020-01-01 00:00:00,20.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,5.0,3,...,0.0,4.1,-6.5,4.1,0,0,1,19.35,1.5,104.2
37370,2020-01-01 00:00:00,20.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,5.0,3,...,0.0,4.1,-6.5,4.1,0,0,1,19.35,1.5,104.2


## 소매업태별(홈쇼핑) 판매액지수(불변지수)

In [53]:
def RS(df = X):
    df['RS'] = 0
    day_list = []
    for date in df.loc[:,'방송일시']:
        year = date[2:4]
        month = date[5:7]
        day_list.append(year+month)
    df['연월'] = day_list
    x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13 = 139.3, 113.3, 134.0, 130.1, 133.8, 123.4, 124.7, 120.3, 124.3, 139.0, 138.8, 138.0, 131.7
    df.loc[df['연월']== '1901','RS'] = x1
    df.loc[df['연월']== '1902','RS'] = x2
    df.loc[df['연월']== '1903','RS'] = x3
    df.loc[df['연월']== '1904','RS'] = x4
    df.loc[df['연월']== '1905','RS'] = x5
    df.loc[df['연월']== '1906','RS'] = x6
    df.loc[df['연월']== '1907','RS'] = x7
    df.loc[df['연월']== '1908','RS'] = x8
    df.loc[df['연월']== '1909','RS'] = x9
    df.loc[df['연월']== '1910','RS'] = x10
    df.loc[df['연월']== '1911','RS'] = x11
    df.loc[df['연월']== '1912','RS'] = x12
    df.loc[df['연월']== '2001','RS'] = x13
    del df['연월']
    
    return(df)

In [54]:
X = RS(X); X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균방송분,겹치는수,...,평균기온(℃),최저기온(℃),최고기온(℃),연예인,월급날,휴일,상품군power,CPI,CSI,RS
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,10.0,1,...,-2.1,-5.8,2.1,0,0,1,26.38,0.8,97.5,139.3
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,10.0,1,...,-2.1,-5.8,2.1,0,0,1,26.38,0.8,97.5,139.3
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,10.0,2,...,-2.1,-5.8,2.1,0,0,1,26.38,0.8,97.5,139.3
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,10.0,2,...,-2.1,-5.8,2.1,0,0,1,26.38,0.8,97.5,139.3
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,10.0,3,...,-2.1,-5.8,2.1,0,0,1,26.38,0.8,97.5,139.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37367,2019-12-31 23:40:00,20.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,5.0,2,...,-3.8,-6.7,0.4,0,0,0,19.35,0.7,100.5,138.0
37368,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,5.0,3,...,4.1,-6.5,4.1,0,0,1,19.35,1.5,104.2,131.7
37369,2020-01-01 00:00:00,20.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,5.0,3,...,4.1,-6.5,4.1,0,0,1,19.35,1.5,104.2,131.7
37370,2020-01-01 00:00:00,20.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,5.0,3,...,4.1,-6.5,4.1,0,0,1,19.35,1.5,104.2,131.7


## 시청률 data

주파수를 사용하지 않는 케이블, 위성방송, IPTV의 등장에 따라 셋톱박스에서 조사가 가정해져서 시청률이 좀 더 정확해졌으며, 시청률 조사지역은 점차 확대되어 수도권, 5대 광역시, 중소도시등이 포함되어 2019년에는 모든 케이블 방송권역에서 조사가 이루어지고 있다. 2020년 현재 TNMS는 전국(제주도 및 읍·면 지역 포함) 4,000여 가구에 거주하는 약 9천명을 셋탑박스를 조사(TMNS 홈페이지 기준), 닐슨 코리아에서는 피플미터기로 4,220가구(닐슨 컴퍼니 코리아 홈페이지 기준)를 조사한다.

+) http://www.mediaus.co.kr/news/articleView.html?idxno=22809

In [55]:
# 시청률 데이터파일(주최측에서 제공한) 의 이름을 Train_rating 으로 변경해 주세요

In [56]:
# 앞의 과정처럼 그 폴더 안의 Train set.xlsx 을 읽습니다. 
rate_df=pd.read_excel(directory+'/Train_rating.xlsx',header = 1) 

In [57]:
# 데이터 reset 하고싶으면 위의 셀 말고 이거 돌려가면서 reset 합시다.(앞과 같은 과정입니다)
rate = rate_df.copy()

In [58]:
rate

,시간대,2019-01-01,2019-01-02,2019-01-03,2019-01-04,2019-01-05,2019-01-06,2019-01-07,2019-01-08,2019-01-09,...,2019-12-23,2019-12-24,2019-12-25,2019-12-26,2019-12-27,2019-12-28,2019-12-29,2019-12-30,2019-12-31,2019-01-01 to 2019-12-31
0,02:00,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.003
1,02:01,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.012,0.003
2,02:02,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.004
3,02:03,0.000,0.000,0.014,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.017,0.000,0.000,0.000,0.000,0.000,0.000,0.004
4,02:04,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.017,0.000,0.000,0.000,0.000,0.000,0.000,0.004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1436,01:56,0.000,0.000,0.000,0.000,0.027,0.000,0.000,0.000,0.000,...,0.013,0.000,0.000,0.000,0.015,0.000,0.000,0.000,0.000,0.004
1437,01:57,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.004,...,0.013,0.000,0.000,0.000,0.015,0.000,0.000,0.000,0.000,0.004
1438,01:58,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.004,...,0.013,0.017,0.000,0.000,0.015,0.000,0.000,0.019,0.000,0.004
1439,01:59,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.004,...,0.013,0.000,0.000,0.000,0.015,0.000,0.000,0.000,0.000,0.004


In [59]:
onemin_dict=rate['시간대'].to_dict()
onemin_dict[1440] = '일총합'

In [60]:
rate=rate.rename(index=onemin_dict)
rate.drop(['시간대'], axis=1, inplace=True)
rate

,2019-01-01,2019-01-02,2019-01-03,2019-01-04,2019-01-05,2019-01-06,2019-01-07,2019-01-08,2019-01-09,2019-01-10,...,2019-12-23,2019-12-24,2019-12-25,2019-12-26,2019-12-27,2019-12-28,2019-12-29,2019-12-30,2019-12-31,2019-01-01 to 2019-12-31
02:00,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.003
02:01,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.005,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.012,0.003
02:02,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.005,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.004
02:03,0.000,0.000,0.014,0.000,0.000,0.000,0.000,0.000,0.000,0.005,...,0.000,0.000,0.017,0.000,0.000,0.000,0.000,0.000,0.000,0.004
02:04,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.005,...,0.000,0.000,0.017,0.000,0.000,0.000,0.000,0.000,0.000,0.004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
01:56,0.000,0.000,0.000,0.000,0.027,0.000,0.000,0.000,0.000,0.000,...,0.013,0.000,0.000,0.000,0.015,0.000,0.000,0.000,0.000,0.004
01:57,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.004,0.000,...,0.013,0.000,0.000,0.000,0.015,0.000,0.000,0.000,0.000,0.004
01:58,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.004,0.000,...,0.013,0.017,0.000,0.000,0.015,0.000,0.000,0.019,0.000,0.004
01:59,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.004,0.000,...,0.013,0.000,0.000,0.000,0.015,0.000,0.000,0.000,0.000,0.004


In [61]:
# 시청률 data 처리
# 0을 20분 간격 min 값으로 대체 후 평균 값 산출

def rate1(df = rate):
    for i in range(len(df.columns)-1):
        for j in range(72):
            rate_unit = df.iloc[20*j:20*(j+1), i]
            rate_unit = rate_unit.replace(0, rate_unit[rate_unit>0].min() * 0.01)
            rate_unit.iloc[0,] = rate_unit.mean()
            rate_unit.iloc[1:20,] = 0
            df.iloc[20*j:20*(j+1), i] = rate_unit
    
    #20분당일년평균 시청률 자료 처리
    for k in range(72):
        rate_year = df.iloc[20*k:20*(k+1), -1]
        rate_year = rate_year.replace(0, rate_year[rate_year>0].min() * 0.01)
        rate_year.iloc[0,] = rate_year.mean()
        rate_year.iloc[1:20,] = 0
        df.iloc[20*k:20*(k+1), -1] = rate_year
    df = df.fillna(0)
    return df

rate = rate1(rate)
rate

,2019-01-01,2019-01-02,2019-01-03,2019-01-04,2019-01-05,2019-01-06,2019-01-07,2019-01-08,2019-01-09,2019-01-10,...,2019-12-23,2019-12-24,2019-12-25,2019-12-26,2019-12-27,2019-12-28,2019-12-29,2019-12-30,2019-12-31,2019-01-01 to 2019-12-31
02:00,0.000833,0.000,0.000833,0.001526,0.000,0.000,0.000,0.000,0.000,0.003875,...,0.00,0.001781,0.013634,0.000,0.000654,0.000,0.000,0.000,0.000714,0.0032
02:01,0.000000,0.000,0.000000,0.000000,0.000,0.000,0.000,0.000,0.000,0.000000,...,0.00,0.000000,0.000000,0.000,0.000000,0.000,0.000,0.000,0.000000,0.0000
02:02,0.000000,0.000,0.000000,0.000000,0.000,0.000,0.000,0.000,0.000,0.000000,...,0.00,0.000000,0.000000,0.000,0.000000,0.000,0.000,0.000,0.000000,0.0000
02:03,0.000000,0.000,0.000000,0.000000,0.000,0.000,0.000,0.000,0.000,0.000000,...,0.00,0.000000,0.000000,0.000,0.000000,0.000,0.000,0.000,0.000000,0.0000
02:04,0.000000,0.000,0.000000,0.000000,0.000,0.000,0.000,0.000,0.000,0.000000,...,0.00,0.000000,0.000000,0.000,0.000000,0.000,0.000,0.000,0.000000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
01:56,0.000000,0.000,0.000000,0.000000,0.000,0.000,0.000,0.000,0.000,0.000000,...,0.00,0.000000,0.000000,0.000,0.000000,0.000,0.000,0.000,0.000000,0.0000
01:57,0.000000,0.000,0.000000,0.000000,0.000,0.000,0.000,0.000,0.000,0.000000,...,0.00,0.000000,0.000000,0.000,0.000000,0.000,0.000,0.000,0.000000,0.0000
01:58,0.000000,0.000,0.000000,0.000000,0.000,0.000,0.000,0.000,0.000,0.000000,...,0.00,0.000000,0.000000,0.000,0.000000,0.000,0.000,0.000,0.000000,0.0000
01:59,0.000000,0.000,0.000000,0.000000,0.000,0.000,0.000,0.000,0.000,0.000000,...,0.00,0.000000,0.000000,0.000,0.000000,0.000,0.000,0.000,0.000000,0.0000


In [62]:
# 시청률 data와 기존 data 병합
# 20분당당일평균시청률: 특정 날짜 특정 20분 동안의 평균 시청률
# 일평균시청률: 특정 날짜의 전체 평균 시청률
# 20분당일년평균시청률: 일년 동안 특정 20분 동안의 평균 시청률
def rate_merge(df1 = rate, df2 = X):        
    df2['20분당당일평균시청률']=0
    df2['일평균시청률']=0
    df2['20분당일년평균시청률']=0
    for i in range(len(df1.columns)-1):    
        rate_unit = df1.iloc[:,i]
        rate_unit = rate_unit[rate_unit>0]
        for j in range(len(rate_unit)):
            df2.loc[df2['시분']==rate_unit.index[j],'20분당당일평균시청률'] = rate_unit.loc[rate_unit.index[j]]
    
    rate_day = df1.iloc[-1,:]
    rate_day = rate_day.transpose()
    for k in range(len(rate_day)):
        df2.loc[df2['날짜']==rate_day.index[k],'일평균시청률'] = rate_day.loc[rate_day.index[k]]
    
    rate_year = df1.iloc[:,-1]
    for m in range(len(df1)-1):
        df2.loc[df2['시분']==rate_year.index[m],'20분당일년평균시청률'] = rate_year.loc[rate_year.index[m]]
    
    df = df2.fillna(0)
    return df

X = rate_merge(rate,X)
X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균방송분,겹치는수,...,연예인,월급날,휴일,상품군power,CPI,CSI,RS,20분당당일평균시청률,일평균시청률,20분당일년평균시청률
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,10.0,1,...,0,0,1,26.38,0.8,97.5,139.3,0.001309,0.004,0.00215
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,10.0,1,...,0,0,1,26.38,0.8,97.5,139.3,0.001309,0.004,0.00215
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,10.0,2,...,0,0,1,26.38,0.8,97.5,139.3,0.001012,0.004,0.00200
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,10.0,2,...,0,0,1,26.38,0.8,97.5,139.3,0.001012,0.004,0.00200
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,10.0,3,...,0,0,1,26.38,0.8,97.5,139.3,0.002180,0.004,0.00225
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37367,2019-12-31 23:40:00,20.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,5.0,2,...,0,0,0,19.35,0.7,100.5,138.0,0.020648,0.005,0.00460
37368,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,5.0,3,...,0,0,1,19.35,1.5,104.2,131.7,0.026600,0.000,0.00460
37369,2020-01-01 00:00:00,20.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,5.0,3,...,0,0,1,19.35,1.5,104.2,131.7,0.026600,0.000,0.00460
37370,2020-01-01 00:00:00,20.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,5.0,3,...,0,0,1,19.35,1.5,104.2,131.7,0.026600,0.000,0.00460


## 심야 시간대 표현

00 시 이후는 24를 더해주었습니다.

In [63]:
X.loc[X.loc[:,'시각'] < 3,'시각'] = X.loc[X.loc[:,'시각'] < 3,'시각'] + 24

In [64]:
X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균방송분,겹치는수,...,연예인,월급날,휴일,상품군power,CPI,CSI,RS,20분당당일평균시청률,일평균시청률,20분당일년평균시청률
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,10.0,1,...,0,0,1,26.38,0.8,97.5,139.3,0.001309,0.004,0.00215
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,10.0,1,...,0,0,1,26.38,0.8,97.5,139.3,0.001309,0.004,0.00215
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,10.0,2,...,0,0,1,26.38,0.8,97.5,139.3,0.001012,0.004,0.00200
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,10.0,2,...,0,0,1,26.38,0.8,97.5,139.3,0.001012,0.004,0.00200
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,10.0,3,...,0,0,1,26.38,0.8,97.5,139.3,0.002180,0.004,0.00225
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37367,2019-12-31 23:40:00,20.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,5.0,2,...,0,0,0,19.35,0.7,100.5,138.0,0.020648,0.005,0.00460
37368,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,5.0,3,...,0,0,1,19.35,1.5,104.2,131.7,0.026600,0.000,0.00460
37369,2020-01-01 00:00:00,20.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,5.0,3,...,0,0,1,19.35,1.5,104.2,131.7,0.026600,0.000,0.00460
37370,2020-01-01 00:00:00,20.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,5.0,3,...,0,0,1,19.35,1.5,104.2,131.7,0.026600,0.000,0.00460


In [65]:
X.columns

Index(['방송일시', '노출(분)', '마더코드', '상품코드', '상품명', '상품군', '판매단가', '취급액', '평균방송분',
       '겹치는수', '요일/시간', '날짜', '년', '월', '일', '시각', '시분', '점심', '저녁', '강수량(mm)',
       '평균기온(℃)', '최저기온(℃)', '최고기온(℃)', '연예인', '월급날', '휴일', '상품군power', 'CPI',
       'CSI', 'RS', '20분당당일평균시청률', '일평균시청률', '20분당일년평균시청률'],
      dtype='object')

## Prime Time

In [66]:
# https://www.etnews.com/20180718000049?m=1

In [67]:
## one-hot vector(프라임타임 or not)

def prime_time(df = X):
    df['PrimeTime'] = 0
    
    for i in range(len(df)):
        if (9 <= df.loc[i,'시각']) and (df.loc[i,'시각'] < 11):
            df.loc[i,'PrimeTime'] = 1
        elif df.loc[i,'상품군']=="농수축" and (16<=df.loc[i,'시각']) and (df.loc[i,'시각']< 18):
            df.loc[i,'PrimeTime'] = 1            
        elif df.loc[i,'상품군']!="농수축" and (20<=df.loc[i,'시각']) and (df.loc[i,'시각']< 23):
            df.loc[i,'PrimeTime'] = 1
    
    return df

In [68]:
prime_time(X) ;X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균방송분,겹치는수,...,월급날,휴일,상품군power,CPI,CSI,RS,20분당당일평균시청률,일평균시청률,20분당일년평균시청률,PrimeTime
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,10.0,1,...,0,1,26.38,0.8,97.5,139.3,0.001309,0.004,0.00215,0
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,10.0,1,...,0,1,26.38,0.8,97.5,139.3,0.001309,0.004,0.00215,0
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,10.0,2,...,0,1,26.38,0.8,97.5,139.3,0.001012,0.004,0.00200,0
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,10.0,2,...,0,1,26.38,0.8,97.5,139.3,0.001012,0.004,0.00200,0
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,10.0,3,...,0,1,26.38,0.8,97.5,139.3,0.002180,0.004,0.00225,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37367,2019-12-31 23:40:00,20.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,5.0,2,...,0,0,19.35,0.7,100.5,138.0,0.020648,0.005,0.00460,0
37368,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,5.0,3,...,0,1,19.35,1.5,104.2,131.7,0.026600,0.000,0.00460,0
37369,2020-01-01 00:00:00,20.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,5.0,3,...,0,1,19.35,1.5,104.2,131.7,0.026600,0.000,0.00460,0
37370,2020-01-01 00:00:00,20.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,5.0,3,...,0,1,19.35,1.5,104.2,131.7,0.026600,0.000,0.00460,0


In [69]:
X['PrimeTime'].sum()
## 생각보다 해당되는 항목이 많네요..!!
## 시간 처리할 때 ( , ) 이 아니라 [ , ) 로 했습니다.

14570

## 남성, 여성 상품

In [70]:
def gender(df=X):
    df["남성상품"]=0
    df["여성상품"]=0
    list_male=["남성","드로즈","런닝","언더셔츠","트렁크"]
    list_female=["여성","브라","밍크","팬티","치마","레깅스","란쥬","핸드백",
                 "앙상블","엔셀라두스 밴딩팬츠","레이스","엘리자베스아덴","버켓 백",
                 "크로스바디 백","립스틱","쿠션","사첼백","숄더","속눈썹","토트백",
                 "원피스","틴트","투웨이백","이지탑","크로스백","클러치백"]
    for i in ["의류","속옷","잡화","이미용"]:
        for j in list_male:
            df.loc[(df["상품군"]==i)&(df.상품명.str.contains(j)),"남성상품"]=1
        for k in list_female:
            df.loc[(df["상품군"]==i)&(df.상품명.str.contains(k)),"여성상품"]=1
    return df

In [71]:
X = gender(X); X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균방송분,겹치는수,...,상품군power,CPI,CSI,RS,20분당당일평균시청률,일평균시청률,20분당일년평균시청률,PrimeTime,남성상품,여성상품
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,10.0,1,...,26.38,0.8,97.5,139.3,0.001309,0.004,0.00215,0,1,0
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,10.0,1,...,26.38,0.8,97.5,139.3,0.001309,0.004,0.00215,0,0,1
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,10.0,2,...,26.38,0.8,97.5,139.3,0.001012,0.004,0.00200,0,1,0
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,10.0,2,...,26.38,0.8,97.5,139.3,0.001012,0.004,0.00200,0,0,1
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,10.0,3,...,26.38,0.8,97.5,139.3,0.002180,0.004,0.00225,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37367,2019-12-31 23:40:00,20.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,5.0,2,...,19.35,0.7,100.5,138.0,0.020648,0.005,0.00460,0,0,0
37368,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,5.0,3,...,19.35,1.5,104.2,131.7,0.026600,0.000,0.00460,0,0,0
37369,2020-01-01 00:00:00,20.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,5.0,3,...,19.35,1.5,104.2,131.7,0.026600,0.000,0.00460,0,0,0
37370,2020-01-01 00:00:00,20.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,5.0,3,...,19.35,1.5,104.2,131.7,0.026600,0.000,0.00460,0,0,0


## 무이자, 일시불 처리

무이자, 일시불이 소비에 영향을 미칠 것이라 판단하여 상품 별 무이자, 일시불을 나타내는 열 삽입

In [72]:
print('dlf\)')

dlf\)


In [73]:
def pay(df=X):
    df["무이자"]=0
    df["일시불"]=0
    pay_in_inst=["무)", "무이자"]
    pay_in_full=["일)", "일시불"]
    for i in pay_in_inst:
        df.loc[(df.상품명.str.contains(i,regex=False))&(~df.상품명.str.contains("보험")),"무이자"]=1
    for j in pay_in_full:
        df.loc[(df.상품명.str.contains(j, regex=False))&(~df.상품명.str.contains("세일")),"일시불"]=1
    return df

In [74]:
X = pay(X); X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균방송분,겹치는수,...,CSI,RS,20분당당일평균시청률,일평균시청률,20분당일년평균시청률,PrimeTime,남성상품,여성상품,무이자,일시불
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,10.0,1,...,97.5,139.3,0.001309,0.004,0.00215,0,1,0,0,0
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,10.0,1,...,97.5,139.3,0.001309,0.004,0.00215,0,0,1,0,0
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,10.0,2,...,97.5,139.3,0.001012,0.004,0.00200,0,1,0,0,0
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,10.0,2,...,97.5,139.3,0.001012,0.004,0.00200,0,0,1,0,0
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,10.0,3,...,97.5,139.3,0.002180,0.004,0.00225,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37367,2019-12-31 23:40:00,20.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,5.0,2,...,100.5,138.0,0.020648,0.005,0.00460,0,0,0,0,1
37368,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,5.0,3,...,104.2,131.7,0.026600,0.000,0.00460,0,0,0,1,0
37369,2020-01-01 00:00:00,20.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,5.0,3,...,104.2,131.7,0.026600,0.000,0.00460,0,0,0,0,1
37370,2020-01-01 00:00:00,20.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,5.0,3,...,104.2,131.7,0.026600,0.000,0.00460,0,0,0,1,0


In [75]:
# 확인용
X.loc[X['무이자']==1, '상품명']

45                 무이자 LG 통돌이 세탁기
47                 무이자 LG 통돌이 세탁기
49                 무이자 LG 통돌이 세탁기
80       무이자 쿠첸 풀스텐 압력밥솥 10인용(A1)
82        무이자 쿠첸 풀스텐 압력밥솥 6인용(A1)
                   ...           
37362               무이자쿠첸압력밥솥 6인용
37364              무이자쿠첸압력밥솥 10인용
37366               무이자쿠첸압력밥솥 6인용
37368              무이자쿠첸압력밥솥 10인용
37370               무이자쿠첸압력밥솥 6인용
Name: 상품명, Length: 7186, dtype: object

## 유명 기업/브랜드명

In [76]:
def brandname(df=X):
    df['유명기업/브랜드']=0
    lst=["LG","SPC삼립","경남제약", "광동", "구찌", "굿프렌드", "농협", "딤채",
         "락앤락","리복","마리끌레르","뱅뱅","베지밀","보루네오","본죽","삼성",
         "엘르","종근당건강","쿠쿠",'쿠첸',"푸마","프라다","하림","한샘"]
    for i in lst:
        df.loc[df.상품명.str.contains(i),'유명기업/브랜드']=1
    for i in range(len(df)):
        if ("폴로" in df.loc[i,"상품명"]) or ("USPA" in df.loc[i,"상품명"]):
            df.loc[i,"상품명"] = 1
            
    return df

In [77]:
X = brandname(X); X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균방송분,겹치는수,...,RS,20분당당일평균시청률,일평균시청률,20분당일년평균시청률,PrimeTime,남성상품,여성상품,무이자,일시불,유명기업/브랜드
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,10.0,1,...,139.3,0.001309,0.004,0.00215,0,1,0,0,0,0
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,10.0,1,...,139.3,0.001309,0.004,0.00215,0,0,1,0,0,0
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,10.0,2,...,139.3,0.001012,0.004,0.00200,0,1,0,0,0,0
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,10.0,2,...,139.3,0.001012,0.004,0.00200,0,0,1,0,0,0
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,10.0,3,...,139.3,0.002180,0.004,0.00225,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37367,2019-12-31 23:40:00,20.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,5.0,2,...,138.0,0.020648,0.005,0.00460,0,0,0,0,1,1
37368,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,5.0,3,...,131.7,0.026600,0.000,0.00460,0,0,0,1,0,1
37369,2020-01-01 00:00:00,20.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,5.0,3,...,131.7,0.026600,0.000,0.00460,0,0,0,0,1,1
37370,2020-01-01 00:00:00,20.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,5.0,3,...,131.7,0.026600,0.000,0.00460,0,0,0,1,0,1


# 필요없는 변수삭제

In [78]:
X.columns

Index(['방송일시', '노출(분)', '마더코드', '상품코드', '상품명', '상품군', '판매단가', '취급액', '평균방송분',
       '겹치는수', '요일/시간', '날짜', '년', '월', '일', '시각', '시분', '점심', '저녁', '강수량(mm)',
       '평균기온(℃)', '최저기온(℃)', '최고기온(℃)', '연예인', '월급날', '휴일', '상품군power', 'CPI',
       'CSI', 'RS', '20분당당일평균시청률', '일평균시청률', '20분당일년평균시청률', 'PrimeTime',
       '남성상품', '여성상품', '무이자', '일시불', '유명기업/브랜드'],
      dtype='object')

In [79]:
X.loc[:,'상품군power'].unique()

In [80]:
#X.drop(columns=['방송일시','마더코드','상품코드','상품명','요일/시간','날짜','년','월','일','시분','상품군power'],inplace=True)

In [81]:
#X

In [83]:
X.to_excel('processed.xlsx')